In [8]:
# Tottenham,Newcastle
# Aston Villa,Leicester
# Bournemouth,Everton
# Crystal Palace,Chelsea
# Manchester City,West Ham
# Southampton,Brentford
# Brighton,Arsenal
# Fulham,Ipswich
# Liverpool,Manchester United
# Wolves,Nottingham Forest

In [9]:
import pandas as pd

pd.read_csv('./E0 24-25.csv')[['WHH', 'WHD', 'WHA']]

,WHH,WHD,WHA
0,1.65,4.2,5.00
1,8.50,5.5,1.33
2,1.18,7.0,17.00
3,2.60,3.5,2.70
4,1.35,5.5,8.00
...,...,...,...
194,2.70,3.6,2.45
195,4.20,3.8,1.80
196,1.57,4.2,5.50
197,1.30,5.8,9.50


In [59]:
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

driver = webdriver.Firefox()
driver.get(premLeague)
matches = driver.find_elements(By.CSS_SELECTOR, "article.sp-o-market--default")[0:10]
details = pd.DataFrame({'h_team':[], 'a_team':[], 'WHH':[], 'WHD':[], 'WHA':[]})

# Get links for the matches
# [[match, link]]
for match in matches:
    # matches_links.append([match.text, match.get_attribute('href')])
    # Extract teams
    teams = match.find_element(By.CSS_SELECTOR, 'main.sp-o-market__title span').text
    odds_els = match.find_elements(By.CSS_SELECTOR, 'section.sp-o-market__buttons .sp-betbutton > span')

    # Extract odds
    odds = [round(int(btn.text.split('/')[0]) / int(btn.text.split('/')[1]) +1, 2)   for btn in match.find_elements(By.CSS_SELECTOR, 'section.sp-o-market__buttons .sp-betbutton > span')]

    odds = []
    for btn in odds_els:
        if btn.text == 'EVS':
            odds.push(round(2.0, 2))
        else:
            odds.append(round(int(btn.text.split('/')[0]) / int(btn.text.split('/')[1]) +1, 2))
    h_odds = 1/odds[0]
    d_odds = 1/odds[1]
    a_odds = 1/odds[2]

    sum_odd_probs = h_odds + d_odds + a_odds
    match_details = {
        'h_team': [teams.split(' v ')[0]],
        'a_team': [teams.split(' v ')[1]],
        'WHH': [round(h_odds/sum_odd_probs, 3)],
        'WHD': [round(d_odds/sum_odd_probs, 3)],
        'WHA': [round(a_odds/sum_odd_probs, 3)]
        }
    match_dets_df = pd.DataFrame(match_details)
    details = pd.concat([details, match_dets_df], ignore_index=True)
    # print(odds)
    # print({'Home': teams.split(' v ')[0], 'Away': teams.split(' v ')[1], 'WHH': odds[0], 'WHD': odds[1], 'WHA': odds[2]  })


driver.close()


details

,h_team,a_team,WHH,WHD,WHA
0,Newcastle,Bournemouth,0.576,0.226,0.198
1,Brentford,Liverpool,0.162,0.187,0.651
2,Leicester,Fulham,0.241,0.251,0.508
3,West Ham,Crystal Palace,0.358,0.284,0.358
4,Arsenal,Aston Villa,0.634,0.220,0.146
5,Everton,Tottenham,0.340,0.264,0.396
6,Man Utd,Brighton,0.456,0.260,0.284
7,Nottingham Forest,Southampton,0.671,0.204,0.125
8,Ipswich,Man City,0.117,0.188,0.695
9,Chelsea,Wolves,0.702,0.172,0.126


In [60]:

# Manchester City,
# Manchester United
def update_name(row):
    if(row['h_team'] == 'Man Utd'):
        print(row['h_team'], row['a_team'])
        row['h_team'] = 'Manchester United'
    elif(row['a_team'] == 'Man Utd'):
        row['a_team'] = 'Manchester United'

    if(row['h_team'] == 'Man City'):
        row['h_team'] = 'Manchester City'

    elif(row['a_team'] == 'Man City'):
        row['a_team'] = 'Manchester City'
    return row


details_ = details.apply(update_name, axis=1)
details_

Man Utd Brighton


,h_team,a_team,WHH,WHD,WHA
0,Newcastle,Bournemouth,0.576,0.226,0.198
1,Brentford,Liverpool,0.162,0.187,0.651
2,Leicester,Fulham,0.241,0.251,0.508
3,West Ham,Crystal Palace,0.358,0.284,0.358
4,Arsenal,Aston Villa,0.634,0.220,0.146
5,Everton,Tottenham,0.340,0.264,0.396
6,Manchester United,Brighton,0.456,0.260,0.284
7,Nottingham Forest,Southampton,0.671,0.204,0.125
8,Ipswich,Manchester City,0.117,0.188,0.695
9,Chelsea,Wolves,0.702,0.172,0.126


In [61]:
details.to_csv('./odds_22.csv')